In [1]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import PIL

#For image segmentation
import skimage.data as data
import skimage.segmentation as seg
import skimage.filters as filters
import skimage.draw as draw
import skimage.color as color


import keras as keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, AveragePooling2D


In [2]:
def image_show(image, nrows = 1, ncols =1, cmap = 'gray'):
    fig, ax = plt.subplots(nrows = nrows, ncols = ncols, figsize = (14,14))
    ax.imshow(image, cmap = 'gray')
    ax.axis("off")
    return fig, ax

In [3]:
def processes_image(path):
    
    for entry in os.listdir(path):
        if os.path.isfile(os.path.join(path, entry)):
            image = os.path.join(path, entry)
            
            
            img = cv2.imread(image)
            
            imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            ret, thresh =  cv2.threshold(imgray, 150, 220, 0)
            
            contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
            
            img = cv2.drawContours(img, contours, -1, (0,255,0), 3)
            
            img_slic = seg.slic(img, n_segments = 140)
            
            img = color.label2rgb(img_slic, img, kind = 'avg')
            cv2.imwrite(image, img)
            
    

In [4]:
Normal_train_path = "./Covid19-dataset/train/Normal"
Covid_train_path = "./Covid19-dataset/train/Covid"
Pneumonia_train_path = "./Covid19-dataset/train/Viral Pneumonia"

Normal_test_path = "./Covid19-dataset/test/Normal"
Covid_test_path = "./Covid19-dataset/test/Covid"
Pneumonia_test_path = "./Covid19-dataset/test/Viral Pneumonia"

In [6]:
processes_image(Normal_train_path)
processes_image(Covid_train_path)
processes_image(Pneumonia_train_path)

processes_image(Normal_test_path)
processes_image(Covid_test_path)
processes_image(Pneumonia_test_path)

In [9]:
train_path = "./Covid19-dataset/train"
test_path = "./Covid19-dataset/test"

train_datagen = ImageDataGenerator(
    rescale = 1/255,
    horizontal_flip = True,
    zoom_range = 0.3
)

test_datagen = ImageDataGenerator(
    rescale = 1/255,
    zoom_range = 0.1,
    fill_mode = "nearest"
)

train_generator = train_datagen.flow_from_directory(
    train_path,
    batch_size = 4,
    class_mode = 'categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    batch_size = 2,
    class_mode = "categorical"
)

Found 251 images belonging to 3 classes.
Found 66 images belonging to 3 classes.


In [24]:
#Creating model
CNN_model = Sequential()

In [25]:
#Building and Configuring model

#1st Convolution Layer
CNN_model.add(Conv2D(16, kernel_size = (3,3), padding = "valid", activation = 'relu'))
CNN_model.add(Conv2D(32, kernel_size = (3,3), padding = "valid", activation = 'relu'))
CNN_model.add(Conv2D(64, kernel_size = (5,5), padding = "same"))
CNN_model.add(MaxPooling2D(pool_size = (2,2), padding = "valid"))

#2nd Convolutional Layer
CNN_model.add(Conv2D(64, kernel_size = (5,5), padding = "same", activation = 'relu'))
CNN_model.add(Conv2D(64, kernel_size = (5,5), padding = "same", activation = 'relu'))
CNN_model.add(Conv2D(128, kernel_size = (3,3), padding = "valid"))
CNN_model.add(MaxPooling2D(pool_size = (3,3)))

#Flattening images into vectors
CNN_model.add(Flatten())

#Dense Layers and output layer
CNN_model.add(Dense(units = 128, activation = "relu"))
CNN_model.add(Dense(units = 64, activation = "relu"))
CNN_model.add(Dense(units = 32, activation = "relu"))
CNN_model.add(Dense(units = 16, activation = "relu"))
CNN_model.add(Dense(units = 3, activation = "softmax"))

In [26]:
CNN_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [27]:
CNN_model.fit(
    train_generator,
    epochs = 10,
    validation_data= test_generator
)

Epoch 1/10
63/63 [==============================] - 60s 948ms/step - loss: 1.0963 - accuracy: 0.4502 - val_loss: 0.9116 - val_accuracy: 0.6515
Epoch 2/10
63/63 [==============================] - 59s 944ms/step - loss: 0.7909 - accuracy: 0.6693 - val_loss: 0.7307 - val_accuracy: 0.6818
Epoch 3/10
63/63 [==============================] - 59s 942ms/step - loss: 0.8633 - accuracy: 0.6335 - val_loss: 1.7173 - val_accuracy: 0.5909
Epoch 4/10
63/63 [==============================] - 59s 943ms/step - loss: 0.7126 - accuracy: 0.7490 - val_loss: 0.6777 - val_accuracy: 0.7727
Epoch 5/10
63/63 [==============================] - 60s 951ms/step - loss: 0.5398 - accuracy: 0.7689 - val_loss: 0.6970 - val_accuracy: 0.7879
Epoch 6/10
63/63 [==============================] - 59s 941ms/step - loss: 0.9221 - accuracy: 0.7649 - val_loss: 0.8249 - val_accuracy: 0.6212
Epoch 7/10
63/63 [==============================] - 59s 941ms/step - loss: 0.7235 - accuracy: 0.6653 - val_loss: 0.4983 - val_accuracy: 0.7727